<h1 align=center><font size = 6>Regression Models with Keras</font></h1>
<p style="text-align:center"><b>Assignment / mini-project</b></p>
<p style="text-align:center">by Goran Stevanovic</p>

# Introduction

In this mini-project, I am building a regression model using the deep learning Keras library, and then experimenting with increasing the number of training epochs and changing number of hidden layers, and observing how changing these parameters impacts the performance of the model.

# Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
<a href="#item30">Import libraries</a>  
<a href="#item31">Download and Clean Dataset</a>  
A. <a href="#item32">Build a baseline model</a>  
B. <a href="#item33">Normalize the data</a>  
C. <a href="#item34">Increate the number of epochs</a>  
D. <a href="#item35">Increase the number of hidden layers</a>  

</font>
</div>

<a id="item30"></a>

# Import libraries

Let's install and import libraries which will be used in this notebook.

In [106]:
# import Pandas and NumPy
import pandas as pd
import numpy as np

In [107]:
# import from sklearn functions used for featuring and evaluation 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [108]:
# install TensorFlow and Kkeras
# (already installed in Anaconda)
#!pip install tensorflow
#!pip install keras

#import Keras
import keras
from keras.models import Sequential
from keras.layers import Dense

<a id="item31"></a>

# Download and Clean Dataset

The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:

1. Cement
2. Blast Furnace Slag
3. Fly Ash
4. Water
5. Superplasticizer
6. Coarse Aggregate
7. Fine Aggregate

Let's download the data and read it into a <em>pandas</em> dataframe.

In [109]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
print(concrete_data.shape)
concrete_data.head()

(1030, 9)


,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


There are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

The first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 

Let's check the dataset for any missing values.

In [110]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [111]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

The target variable y in this problem is the concrete sample strength. Therefore, our predictors X will be all the other columns.

In [112]:
X = concrete_data[concrete_data.columns[concrete_data.columns != 'Strength']] # all columns except Strength
y = concrete_data['Strength'] # Strength column
X_number_of_columns = X.shape[1] # number of prediktors

In [113]:
X.head() 

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [114]:
y.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [115]:
print('Number of predictors is: {}'.format(X_number_of_columns)) 

Number of predictors is: 8


<a id='item32'></a>

# A. Build a baseline model

Use the Keras library to build a neural network with the following:

-	One hidden layer of 10 nodes, and a ReLU activation function
-	Use the adam optimizer and the mean squared error as the loss function.
1.	Randomly split the data into a training and test sets by holding 30% of the data for testing. 
2.	Train the model on the training data using 50 epochs.
3.	Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength.  
4.	Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.
5.	Report the mean and the standard deviation of the mean squared errors.


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.

This function create a model that has one hidden layer, with 10 hidden units, ReLU activaton function, and adam optimizer.

In [116]:
# define metrics function using Sklearn mean_squared_error 
def metric_fn(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)

# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(X_number_of_columns,)))
    #model.add(Dense(10, activation='relu'))
    #model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model, use loss mean_squared_error as metrics as well (no need to use Sklearn)
    #model.compile(optimizer='adam', loss='mean_squared_error', metrics=[metric_fn])
    #model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    return model

Let's call the function now to create our model.

In [117]:
# build the model
model = regression_model()

Next, we will train and test the model. We will leave out 30% of the data for validation and we will train the model using 50 epochs.

It will run for 50 times, and mean error and standard deviation of those runs is reported.

In [118]:
# initialize MSE array
mse_loss_scores = np.array([])

# run for 50 times
for i in range(50):
    # train test split, shuffle default=True, random_state=4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)
    #print ('Train set:', X_train.shape,  y_train.shape)
    #print ('Test set:', X_test.shape,  y_test.shape)
    
    # fit the model
    # this option will provide train-test split, with shuffle before each epoch, but in this example Sklearn is used instead.
    #model.fit(X, y, validation_split=0.3, shuffle = TRUE, epochs=50, verbose=1)
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, verbose=0)
    mse_loss = model.evaluate(X_test, y_test, verbose=0)
    #print(mse_loss)
    mse_loss_scores = np.append(mse_loss_scores, mse_loss)

#mse_loss_scores
print('Mean Squared Error is: {}\nStandard Deviation is: {}'.format(np.mean(mse_loss_scores), np.std(mse_loss_scores)))

Mean Squared Error is: 80.35966133438654
Standard Deviation is: 22.30668454367443


<a id='item33'></a>

# B. Normalize the data

This time we reapeat Part A but use a <b>normalized version of the data</b>. We will investigate how does the mean of the mean squared errors compare to that from Part A (where data is not normalized).

In [119]:
# normalize, use formula calculation (no need to use Sklearn)
#X_norm = StandardScaler().fit(X).transform(X)
X_norm = (X - X.mean()) / X.std()
X_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Next, just as in Part A, we will train and test the model. We will leave out 30% of the data for validation and we will train the model using 50 epochs. However, this time we use <b>normalized version of the data</b>.

It will run for 50 times, and mean error and standard deviation of those runs is reported. 

In [120]:
# initialize MSE array
mse_loss_scores = np.array([])

# run for 50 times
for i in range(50):
    # train test split, shuffle default=True, random_state=4
    X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, random_state=4)
    #print ('Train set:', X_train.shape,  y_train.shape)
    #print ('Test set:', X_test.shape,  y_test.shape)
    
    # fit the model
    # this option will provide train-test split, with shuffle before each epoch, but in this example Sklearn is used instead.
    #model.fit(X, y, validation_split=0.3, shuffle = TRUE, epochs=50, verbose=1)
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, verbose=0)
    mse_loss = model.evaluate(X_test, y_test, verbose=0)
    #print(mse_loss)
    mse_loss_scores = np.append(mse_loss_scores, mse_loss)

#mse_loss_scores
print('Mean Squared Error is: {}\nStandard Deviation is: {}'.format(np.mean(mse_loss_scores), np.std(mse_loss_scores)))

Mean Squared Error is: 56.50785851734742
Standard Deviation is: 38.53744789676742


#### How does the mean of the mean squared errors compare to that from Part A?

- Obviosly, results are better when using normalized data for predictors.

<a id='item34'></a>

# C. Increate the number of epochs

This time we reapeat Part B but use  <b>100 epochs</b>. We will investigate how does the mean of the mean squared errors compare to that from Part B (where 50 epochs were used).

Next, just as in Part B, we will train and test the model. We will leave out 30% of the data for validation. However, this time we will train the model using <b>100 epochs</b>. We use normalized data for predictors.

It will run for 50 times, and mean error and standard deviation of those runs is reported. 

In [121]:
# initialize MSE array
mse_loss_scores = np.array([])

# run for 50 times
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, random_state=4)
    
    # fit the model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, verbose=0)
    mse_loss = model.evaluate(X_test, y_test, verbose=0)
    mse_loss_scores = np.append(mse_loss_scores, mse_loss)

print('Mean Squared Error is: {}\nStandard Deviation is: {}'.format(np.mean(mse_loss_scores), np.std(mse_loss_scores)))

Mean Squared Error is: 34.88951501852486
Standard Deviation is: 0.41521888577604843


#### How does the mean of the mean squared errors compare to that from Step B?

- Usign more epochs (=100) provides beeter result then results in Part B.
- Obviosly, using normalized data and more epochs is providing better results

<a id='item35'></a>

# D. Increase the number of hidden layers

This time we reapeat Part B but use <b>three hidden layers, each of 10 nodes and ReLU activation function</b>. We will investigate how does the mean of the mean squared errors compare to that from Part B.

Let's define new function that defines our regression model for us so that we can conveniently call it to create our model.

New function create a model that has three hidden layers, with 10 hidden units, ReLU activaton function, and adam optimizer.

In [122]:
# define regression model with 3 hiden layers
def regression_model_3():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(X_number_of_columns,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model, use loss mean_squared_error as metrics as well (no need to use Sklearn)
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    return model

Now we call function to build the model

Next, using new model, just as in Part B, we will train and test the model. We will leave out 30% of the data for validation and we will train the model using 50 epochs. Again, we use <b>normalized version of the data</b>.

It will run for 50 times, and mean error and standard deviation of those runs is reported. 

In [123]:
# build the model
model = regression_model_3()

# initialize MSE array
mse_loss_scores = np.array([])

# run for 50 times
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, random_state=4)
    
    # fit the model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, verbose=0)
    mse_loss = model.evaluate(X_test, y_test, verbose=0)
    mse_loss_scores = np.append(mse_loss_scores, mse_loss)

print('Mean Squared Error is: {}\nStandard Deviation is: {}'.format(np.mean(mse_loss_scores), np.std(mse_loss_scores)))

Mean Squared Error is: 43.80941148381404
Standard Deviation is: 15.558931526849824


#### How does the mean of the mean squared errors compare to that from Step B?

- MSE is now (with more hidden layers) better then in Part B.
- Standard Deviation is now (with more hidden layers) better then in Part B.
- It is interesting to note, that results in Part C, with one hidden layer, but more epoch, is better then results in Part D with more hidden layers, but less spochs.